# Black Magic AI Detectron2 Panoptic Segmentation Cloud Vision API Tutorial

<img src="../images/blackmagicailogo.png">

This tutorial demonstrats how to create an AWS Detectron2 Panoptic Segmentation Cloud API by deploying a pre-trained  Detectron2 model to an AWS Sagemaker endpoint and exposing it as a REST API using AWS API Gateway.

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.

# Setup required python version to run detectron2 install

### Run these commands in a jupyter notebook terminal

In [ ]:
# This notebook requires python 3.9
# Ref: https://awstip.com/how-to-use-a-newer-python-version-in-aws-sagemaker-notebook-1682a89625ef
conda create -n my_custom_python_36 python=3.6
cd ~/SageMaker
source activate my_custom_python_36
pip install virtualenv
virtualenv my_custom_python_36_venv
conda deactivate
source my_custom_python_36_venv/bin/activate
pip install ipykernel
python -m ipykernel install --user --name=my_custom_python_36

### Install pytorch
pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
### Install openCV (takes a long time to install. Be patient)
pip install opencv-python
### Install Detectron2
pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html
### Install Boto3
pip install boto3
### Install sagemaker
pip install sagemaker

Restart notebook and select new my_custom_python_36 Kernel for this notebook

# Install imports

In [ ]:
# import some common libraries
import numpy as np
import cv2, json
import torch, torchvision
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer

In [ ]:
# check pytorch installation: 
import sys
from platform import python_version
#print(torch.__version__, torch.cuda.is_available())
print(torchvision.__version__)
print(sys.version_info)
print(python_version())

# Run a pre-trained Panoptic Segmentation model
["...panoptic segmentation combines semantic and instance segmentation such that all pixels are assigned a class label and all object instances are uniquely segmented."](https://kharshit.github.io/blog/2019/10/18/introduction-to-panoptic-segmentation-tutorial)

Panoptic segmentation classifies all pixels in the image within a polygonal boundaing area including objects and background scenery. Unlike, object and Instance segmentation which only care about individual objects in the image.



Define source Image

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageCms

image_filename="city-scene.jpg"
input_image="../images/" + image_filename

image_src = Image.open(input_image)
np_image = np.array(image_src, dtype='float32')

image_src.show()

profile = ImageCms.createProfile("XYZ")
image_src.save("../images/abc.jpg", icc_profile=ImageCms.ImageCmsProfile(profile).tobytes());

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [ ]:
# Step 1
cfg = get_cfg()

Define custom Panoptic visualizer

In [ ]:
def pan_seg_visualizer(predictionsSegs, instance_list, image_src, stuff_classes, stuff_colors, thing_classes, thing_colors, boxes, opacity):
    imageSize = np.append(tuple(reversed(image_src.size)), 4) # get image shape info
    rgba = np.zeros(imageSize, dtype = np.uint8)
    rgba[:, :] = [255, 255, 255, 0]
    font = ImageFont.truetype('fonts/FreeSerif.ttf', 8)
    stuff_array=[]
    for seg_info in instance_list:
        maskX=(predictionsSegs==seg_info["id"])
        binary_mask=np.array((maskX == True),  dtype=int)
        if (seg_info["isthing"]):
            color=thing_colors[seg_info["category_id"]]
            name=thing_classes[seg_info['category_id']]
            rgba[(binary_mask == 1), :] = np.append(color, opacity)
        else: # isStuff
            name=stuff_classes[seg_info['category_id']]
#             if (name == "road"): # filter specific mask
            color=stuff_colors[seg_info["category_id"]]
            x,y = np.argwhere(binary_mask == 1).mean(axis=0)
            stuff_array.append((name,x,y))
            rgba[(binary_mask == 1), :] = np.append(color, opacity)

    maskXImg = Image.fromarray(np.asarray(rgba),mode='RGBA')
    draw = ImageDraw.Draw(maskXImg)
#     Draw boxes and instance labels
    for seg_info, boxx in zip(instance_list, boxes):
      box=boxx
      text = f"{thing_classes[seg_info['category_id']]} {.85:.0%}"
      len = draw.textlength(text=text)
      bbox = draw.textbbox((box[0], box[1]), text, font=font)
      h = bbox[3] - bbox[1]
      draw.rectangle([(box[0], box[1]-h), (box[0] + len, box[1])], fill=(0,0,0))#text background rectangle
      draw.text((box[0], box[1]-h), text, fill=(255, 255, 255)) # draw text on instance
      draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline=(0,255,0))#blue rectangle
    for stuff in stuff_array:       
        x1=stuff[1]
        y1=stuff[2]
        text=stuff[0]
        len = draw.textlength(text=text)
        width_text, height_text = draw.textsize(text, font)
        draw.rectangle([(y1, x1), (y1 + len, x1+height_text)], fill=(0,0,0))#text background rectangle
        draw.text((y1, x1), text, fill=(255, 255, 255)) # draw text on instance
    return Image.alpha_composite(image_src, maskXImg)
#     return maskXImg # return segmentation mask only

In [ ]:
# Panoptic Segmentation
# Ref: https://youtu.be/Pb3opEFP94U
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)
model.eval()

Use Default Predictor and Visualizer to validate Model

In [ ]:
# Panoptic Segmentation Visualizer
# We can use `Visualizer` to draw the predictions on the image.
predictor = DefaultPredictor(cfg) # normal operation
predictions, segmentInfo = predictor(np_image)["panoptic_seg"]
v = Visualizer(np_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
# Uncomment to filter out specific segments
# out = v.draw_panoptic_seg_predictions(predictions.to("cpu"), list(filter(lambda x: x['category_id'] == 17, segmentInfo)), area_threshold=.1)
out = v.draw_panoptic_seg_predictions(predictions.to("cpu"), segmentInfo, area_threshold=.1)
plt.imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Save image to local file system
# cv2.imwrite('../images/test.png',cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))

### Export Model

In [ ]:
# Export model
torch.save(model, "../models/model-pan.pth", _use_new_zipfile_serialization=True)

In [ ]:
%%bash
# Create model asset required for Sagemaker endpoint deployment. Copy model.tar.gz to S3 bucket model folder.
# Rename model-object.pth file name to model-object.pth per required by Sagemaker endpoint specs
tar --transform='flags=r;s|models/model-pan.pth|model.pth|' -czvf ../models/model.tar-pan.gz ../models/model-pan.pth ../code/inference-pan.py

Load Exported Model and Validate

In [ ]:
# Load model
saved_object_model = torch.load("../models/model-pan.pth")
saved_object_model.eval()

In [ ]:
# Load image input and get predictions
original_image = cv2.imread(input_image) 
# print(original_image.shape)
aug = T.ResizeShortestEdge(
             [800, 800], 1333
#             [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
        )
with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
        # Apply pre-processing to image.
#         if cfg.INPUT.FORMAT == "RGB":
#             # whether the model expects BGR inputs or RGB
#             original_image = original_image[:, :, ::-1]
        height, width = original_image.shape[:2]
        image = aug.get_transform(original_image).apply_image(original_image)
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))

        inputs = {"image": image, "height": height, "width": width}
        predictions = saved_object_model([inputs])
#         print(predictions)

### Use Detectron2 Visualizer on saved model output

In [ ]:
# Panoptic Segmentation Visualizer
# We can use `Visualizer` to draw the predictions on the image.
predictions_pan, segmentInfo = predictions[0]["panoptic_seg"]
v = Visualizer(np_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=.5)
# Uncomment to filter out specific segments
# out = v.draw_panoptic_seg_predictions(predictions.to("cpu"), list(filter(lambda x: x['category_id'] == 17, segmentInfo)), area_threshold=.1)
out = v.draw_panoptic_seg_predictions(predictions_pan.to("cpu"), segmentInfo, area_threshold=.1)
plt.imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

### Display results using custom Panoptic Visualizer which does not use Detectron2 dependances

In [ ]:
# Opening JSON file
with open('pan_metadata.json') as json_file:
    metadata = json.load(json_file)

In [ ]:
image_src = image_src.convert('RGBA')

opacity=150
panoptic_seg=predictions[0]['panoptic_seg'][0].cpu().numpy()
instance_list=predictions[0]['panoptic_seg'][1]
boxes=predictions[0]['instances'].pred_boxes

# Call function
out=pan_seg_visualizer(panoptic_seg, instance_list, image_src, metadata["stuff_classes"], metadata["stuff_colors"], metadata["thing_classes"], metadata["thing_colors"], boxes, opacity)
out.show()

***Save processed image to a file (optional)***

In [ ]:
out.save("../images/" + image_filename + ".png")

### Deploy Panoptic Model to Endpoint

Upload model.tar.gz file to s3 bucket model folder

In [ ]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role, Session

sess = Session(default_bucket='<INSERT-AWS-S3-BUCKET-NAME-HERE>')

# print(model_data)
role = get_execution_role()

# Connect to S3 bucket
s3 = boto3.resource('s3')

In [ ]:
# (optional- DO NOT run this cell if the model exist and was previously uploaded in s3) upload file to s3 bucket
s3.Bucket('<INSERT-AWS-S3-BUCKET-NAME-HERE>').upload_file("../models/model.tar-pan.gz", "model/model.tar.gz")

In [ ]:
uri = sess.list_s3_files(sess.default_bucket(), 'model')
# print(uri)
model_data = sagemaker.s3.s3_path_join('s3://', sess.default_bucket(), uri[0])

In [ ]:
# panoptic detection
region = sess.boto_region_name
serve_image_uri = f"<INSERT-AWS-ELASTIC-CONTAINER-REGISTRY-REPOSITORY-URI-HERE>" ##custom image

pyModel = PyTorchModel(
    entry_point="inference-pan.py",
    source_dir="../code",
    role=role,
    model_data=model_data,
    image_uri=serve_image_uri,
    framework_version="1.10.2",
    py_version="py38"
)

predictorEndpt = pyModel.deploy(instance_type='ml.p3.2xlarge', initial_instance_count=1)

Validate Endpoint - perform inference

In [ ]:
# Ref:
# https://aws.amazon.com/blogs/compute/handling-binary-data-using-amazon-api-gateway-http-apis/
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-payload-encodings.html
# https://aws.amazon.com/premiumsupport/knowledge-center/api-gateway-binary-data-lambda/
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-payload-encodings-configure-with-control-service-api.html
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-payload-encodings-configure-with-console.html
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-mapping-template-reference.html
import boto3
import io, json
from base64 import b64encode,b64decode
from io     import BytesIO
from PIL import Image, ImageDraw, ImageFont
endpoint = '<INSERT_ENDPOINT_NAME_HERE>'

runtime= boto3.client('runtime.sagemaker')

base='/9j/4AAQSkZJRgABAQEAAAAAAAD/2wBDAAoHCAkIBgoJCAkLCwoMDxkQDw4ODx8WFxIZJCAmJiQgIyIoLToxKCs2KyIjMkQzNjs9QEFAJzBHTEY/Szo/QD7/2wBDAQsLCw8NDx0QEB0+KSMpPj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj7/xAAfAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgv/xAC1EAACAQMDAgQDBQUEBAAAAX0BAgMABBEFEiExQQYTUWEHInEUMoGRoQgjQrHBFVLR8CQzYnKCCQoWFxgZGiUmJygpKjQ1Njc4OTpDREVGR0hJSlNUVVZXWFlaY2RlZmdoaWpzdHV2d3h5eoOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4eLj5OXm5+jp6vHy8/T19vf4+fr/xAAfAQADAQEBAQEBAQEBAAAAAAAAAQIDBAUGBwgJCgv/xAC1EQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXxFxgZGiYnKCkqNTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqCg4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2dri4+Tl5ufo6ery8/T19vf4+fr/wAARCAEoAZADASEAAhEBAxEB/9oADAMBAAIRAxEAPwCgOlN7VrufPXKl1/x82/phqguI5P8AhG7zzGAZY2IY8d6pDicxddNMIHW2yffk0grGbPYhsPFPFZlj1qUGpC5ItSqaz2AkFSKazKLUT7a0oJQRWL0Atpg1KFrMY7FLsq0AbaNtSMTbVHU7FbtoJJIJZFTKN9mOJEz0kq4S5WDItIsPGetWtzDb2Nqltv2m5kh2FhUh8Eam1nFaXt+hji7JbtuoliVH4AtHqWYfDGqQWaW1lcRNs6b4CuapwRaz4c1ScXunQxz6hIkFrqV0QYIMrUwq33Jl5My5bWytMtJM2qXr5f7SzNHGue1U3m3E/MDW/vSdw6WIHPGcH601fWtR2GtURrRMgjammruFhhpppXYDOlOEvPNW9RseGzTs8UiSMn5qM0DsM34NNLmgYwvTSad7AJmj5jSA9HFJ/Ca2R4JVm/4+7Mt03sD+VVGiMXhLUE84y/unb5l6fMvFaFwfQ5m6ORpfvbY/nRXOexHYdThzSLJFp461DiSSqakTismMlBp9ZjJUNXImPXNZzRRehmq/FKGrKwFhaftpALsz0qYWNyfu20x/4AaWoXQhsbkfegkX/eXFQtA6ctsX6uBT5gWpk6tqV3oUdrcWVyJVdzGbfeGA75qovjvWmGRJYr7FQa0jSUlcXKmSL481nHzmzb/diGaZ/b9zqVpI0xRSXK43jin7FRBQM5/3nTn6VCUyfu5q0MgkhTOSlR42jArZCYxqjNMkjbrTKoBhptVYBKjaqGM3FaUTjvVWCwu/0ozSAaTTd1IZG0gojbcelOwXJsUtAj0JDR2Nb7Hz5VuP9bZgdXmKj8qrTSK/hjVCu4bYipz9RTNY7o5ef/mGegtv8aKwluexHYcKdmk9C0KtSelQwuPBqVWrNq4D804PWdhonRqso9YSGWlOyMySEJGBks3Sol1qyVtkDS3j5+7boTSUGwJv+EhulOLfSNv+1dT0HXddYfLPpkH+5a7qajGPmFr7laS+1qdSJ9du/m7QAR1W+zK/+uutQlP+1dGrvyfBoU4oadOsG6xTH6ztQulaaP8AlyT8WY0OtPuK3YmFjZ5GLWIY9BTzaWv/AD6w/wDfNSqkh2GG1tf+fSH/AL5pps7RvvW4/Dir9owsMOm2Lfehf/v4ab/ZdgPuxTL9JjTdaYWQf2fD/DLcL/20zTDYv/DdL/wNc0KZNrkDWVyp/wBbDJ+a1E0N0Ofss2PpWqmupJXLjPzcH3o61oAym1dxje9NpCIXqI1YxuCT71Kkcp64A96TGTiMD3qG7H7ofWhCKuant4n3biCBTkDLWKaSAKgFqehL0oHOa6T55lWZP9IsQTx5/wD7LVAXE1x4a1gTp/qoyAcY7iraNKepzt3/AMwsf3bX/Gm96xk7HsU9h+aUVj5ljulPzTsNig08GobELmnK1ZsolV6l85gh2HB9azcbjIfK8yQSTsZnHTd0FWlZtuO3pQ7FDwCTUmKxvqA4ClC0XGPC0/FTcQYpdtIbEK0myi4hNtJimAmKaV4q+YBhFR4p3JGkn+Ln681VltYm55U/7NaqdgaKU0Ri68gng1BXQiGNphNWMi5ZuKcIP7xz9KbGTKoA4p+KzYhCKbsVxhxkUAKoRB8gAoZt1NCIzUdOwz0SNtxwoz9KsRQsTjaefauhaHz1h0tpIWhwh+V/6VS1DL+HtYB3bEtz8x6E5FapFR0Zxt/EFXTGyMmzHSoK55ns03eItOrJo1HUuaT0GOU06p3EwzQHp2KQ4PUiv2rIssoKnUVztgT4p9ZPUdh4WnBakY8CnbaQJC7aXFAhNvFJigY3bSbadxDcU0rV3FYYy1GVpqQEZWo2FaJiZQvvuJ9apV1x2IGU3AJrQCRIgM8UvFSA7Io3UhMQmmVQCU2gBKTFNAdrd2bX0cK+ZCBHJv5B54qzpGnPaXDlpkIYDG0niui76Hh8ysd+uuaeunpZy2bNldpZTzXEasf+KW1TP/PuePxFbS8jOn8Wpwt8SW05MEbLJev1qGuWpue5S2ForLY0H0tTLUpDhS0DExSUNjFFKv3x9azYzVjFWUWuO5VtCYLTwKy6lDwKcE+alcSH7adtpXAXbS7aYg203bTQxNtIVpiYmKYVpWEMIqNlqloJoiZaruK1iIztR+5F/vGqBrthsSFMNWIcspHWnb1NK2ogpM1QDc0lFhBmkzUoYm6k3iq6gegx1NuA4rqPAJop9hAOT+NV3Pm28kTD5HGGHtRYk5TxOoTXkRPu/ZU4rNFZ1T2sM/cFxS4rnubi4p30qQFp1SUhccUmKbKDFOVahjZqxDpVtBXDNlk4Wn7axQDwKeFqmOw4LTsUgHYo21QgxTdtSMQrSbaoQhWmlaBETLTGXiqEQlaryitVqIyNS/1kf0qlXdT2IY2mmrENptMYmfSje1AhDJim+f7U7BYTzqTfmkh2Dd2qRaNh2Olh8Son+s0+Zv8AtsKsDxHZNjfFNH9ecV0Rt1PHlhJrYsR65pr8i7QduQaRdW0kTGZp4xc7Nis6tjGabk0Z/VpoxNcu4NQ1f7VatmLyUTp3FUqyqHp0Y8sbMWlFYbGw6lFHQB4p1Q2ULikxSGGKcBWcmaGtbLlRV6Na4Zy6DsTKvNShKzGOCVJt4pDFC07bV3AXbS7KLiDbSbaYCbabsoATbTWWmSRFKjKUXBkTJVWVauJLRhamf9OI/wBhaqV6UdjMZTTVjG000wEptCAaelQmmikJUijigRKgqSs2BJRWpHUWlWkSKOenSnUnsMWnCoZQ4U8danYELipBSYwxRSeqLFxTsViWbNqvA+lXkSuGe5ROq1IFrMBwWpAtUAoSl20xjtlLtpAIVpNnFMQmyk20AJtphWqEMK1Gy0mIhZaqSrVQBnOap/yFJfYKP0qnXpwMRtNxVgJTTV2EJTaBoaaiIoGKibqnC0AKOKfx3pdQY7FLVtkIKXvU+Yh2KXB70gHCnVMih9OqRju1SVmAtIOtQyx2Kei1Lfumi2NqzGYI/pWgiVwPcpE4WpAtJMByrUgSkMdspQlO4h+yjZTuITZSbabGNK0FKBDdtMKUAMKUwrTuIgdaqyLlsVRJyV+c6ldH/bxVavUh8JixpptaJDEppqrANpDUsBpqNhTGSQ/cqSkAU3mmgRPRT9SBe9OpMQE1B9o5PehRBai/avanface1HIWAuvp9Kf9rGaXswQ8Xa+1OF59D+NRyDJBdr7U4XKGs5RKRJ5y9iKkV8VlylJnQ6T81jC3+zWqgrzpL3jQkVakC0gJFSpQtDAXbTttAC7aNtAgK0m2gY3bSbaYCbaYyU0IjK1Gy0wIXWq+3Myj3qlqQzhHO+aZz1aRv50zvXrLYyG02ruIbTaLgJTTQAlMamUOjPFSUAFJSBFjtSVRmLS0xEbVS/ipdLlx0NS10f7VGrwzSucDcEtWO01ZTwzfTFhFFdOR/wBObVzOu+wOUVuyT/hEdU258m4B9DbNVWXw9fxsVdSp/wBqPFVHE36Ee1j0IDpV0p/g/E4pv2C6HOIv+/lbe0iV7RDfsdwe0f8A38pDazjsn/fVTzRNOYb5Mvpx7GgQS9oZPypc0UNanceFoyvhm0DDDb5M/nW7GteLL4mbkwWpAtZ3AkC1IFpiHbaXbQMXFLtoEJtpNtMYm2m4oEJimFaoQwiomFAEDrUG0eaKcdyWedocgn1Zj+tLXrxMRtNq2IbTTQMQ0yhCEpj1Vyhkf3qsjpQwDikzQJXJ6UUPci+otFILjJBxVA/eq1qhotyblsoWViM8HacVB5su3b50uPTeaIoqye45TOzfI07N7E1IYr0/eFw2P9uk2hckSD/gR496kVm7MaXKiicSS/8APRqlikvHbbC87n0WsZxiWidn1SEZka7iX3HFNW8vf+fyXH4VkoQaKudh4YZpfD8TSNuYTyr+orcjFeZL4jUnC1IBUCJFFSAUALinY4pDFxS7aAE20mKYCYpuKYDdtNIpkkZFRMKaYiCQVRuz5dtcSf8APOJm/SriJnnyD90nrilr1lsYSGU01SENppqwG0lAxKjegAjHGalpSKEpMYpibLNOxQZi0VAiN+lUm4ciriMtyoRpUbH/AGar28TTS7FUnuaFI0NONSgwDxSyiQRGQEYXmseoiOWNZra2eX5Cw4PTmqGCjYYYYVpHsBbtIhNLh22ooyxrXVEG0G6iG3/nlCa56rdy0S/6PGeEnkJ4zK/FVb2xYRG5ijKooy69OPWsYTaeprbQ6Hwcd2gt/s3b/wBK6OMVxVfjKROoqQVmBKoqQCkMcBTsUhi4pcUxCYoIoGM20mKYhuKYRQIjIqIiqEQSCsjXDt0LUj/07n+la09yJbHDdABTTXrGDGU2qASmUwENNpiEqBjzQWTR9KkpDEpKaJLI5op2IHUdqQmiNqoSf61vrVR2KjoXpH3aKvtsquMiBGBPzE9PaojaxcSdJz5TFnO/sKmS83RCMxHOMFqXKOxWAby1twox5nFSXaESK3r8v5VPUZcslxZr6uS1TSq+z5AKxbK5RI/Ot3zCcHpkVoW4H2iPz8SDd824ZzWM9DQ0/BqeTZananrDe/piunjriq/EEScVIKzGSrUq0hj8UuKkYuKQ1QhQKKAGGm0DEIqM0xETVGaZJXkrC8THb4dvv9tVj/M1rS+ImRxVNNesYDDTTVgNpKCRtJVIY01GFBzmhbDHx8VLSsMSmmmIs0U9iB1FQBG9UpP9YauIFzGdDZvRsUtrGJLSPOfl3Y/OoWhaH+ShO4g7qakTSoGjouWESlb0q/VGp+of6mH/AH/6VH2gZasmH2GD2XFTJKjSFSeVrn1ZcR74CnHXtUkR8yPNZyWhojc8OsDrOuov3R5L/pXSR1xVNxlhakFZiJFqUUih4p1IYtIaaJCimMaaaaYDajNIRG1RGqEV5a5vxc23Q0T/AJ63Kj8ua2pbkM5E1HXrWMBp4ppqhDaSqasIbSUhjajDYanYBY2yaloGgppoEWu9LStckM06kwI36VQk/wBZVrYImhH/AMgNwBn5Wb/x6o7GTCuvoc1jHVM0ZLNcJ5T7T82CPpTVaWGONVfaMZK1fIA633NNLM5yWNRXv+tVfTmktwLFicwmMfwfN+FP3+VNn86xa1NEyeS5EigKhFWtMCAx+Y4ij3cuTjFZyWhqaHguYz6jrMx/5awrJ/5Err4jXBWXviRZSpawKHipRTGSCnUhi0UCEooAaaaaAGmmGqERMaiagRWlrlfGJ/0PTx/08Of/AByumh8RlM5emmvTMRhptUA2mmq3FcSm0hid6bVDHZFJuosIN1M3UIC9RQIdRS8hEb9KpTffrRAi2gk/slm3fJ8w/WqaMVbg4qY2sayJ0QzHCfx9auFN/wApBz+tKYrjSwtVCdG6hTz+NUycsSTknmlBDJIXKuGHbtWjDtuuIwS/93HNZ1dBrcvDR75BvlspkQDOXIAqtcCcxmOAKqngscVzwqRkVK5peEA1tqd4sm397bYG0+jCuxiauKv8dy4ltGqUGucokU1MpoKHA07NSMXNAPFMQZpN1ACE00mmAwmmE0IRExqJjVCKsrcVyfi9v31hH6b3/pXRQ+Mymc7TDXpmQw001YhtN3CqAZuFN30WATdTS4oYDS9JvoGN30b6dgsaopOaRI4e1IetIkY/SqM3+sqkNG5Z4PhScd8S/wBKw6iktzRgOf8A61WBBdkY8uf86u6W4EbRmI7XQofRhigUkA8Ve06O4u5WjtovNYDJyeBWdV2V2UjcXw/qGR8tu7f3fMqjL5sLKlxE0TsMgNXDGabGNhvptPukuoAjMn8Djhh6V0dn4t0mUlblLiyPXLfOtFbD8/vRGpGtDrGjytiHWLJvq+yrv2hAN3mIV9VYGuKVOUdGXzdSpNr9tCcKjzH/AGTirOn61bXx2qGhk/uOc1Xs9AUjSDU8NWJpcXdxQG4piFzSbqChN1QxzpLuMbZ2ttP1p2JHFqjJpiE2SHojH8Kr3H7hcz4iHrIdtMVzFv8AW9Ot4XKXsM0oHCQtuNcddXMt15ZmOSgIruoUre8zFyuVjTGrtIGU00xEchwh+lUvMNWtihN5pMmrATJoJpAJmkoGLg0bWpMDYpaRlcX6UflUiI2qlc/fHuK0iNI0bBi2izxrn7zfyrNHJ471C6mhaj2xI4/iPf0p280WKSBSCMHkVEy7W74PSi4pJofFCZ5BGCFH8Tk/dFdpprWap9n0+dJFxzhCM1xYptrQaRqpuHUGob2CDUbb7NMrA/8ALKQD7jetcCepXKcPc5VZEkBDo21sj3qoH2HfnoyH9a9VfwxLc9uvWsJHkiks7KVCuPmtk9KZa+FfCGoW4lu9CiEncwyvGK8qFSVJ+6aS1Wo6f4ceF9n7m61W0C9BDd1yt54Xjt/EEtna6/eRiNUZWmhV/vCuhV4vpczd+hv6L4Y1m5aWG18XxfuQp/5BKN1zW2vhDxAOvia0f66QorenTp1IXUA23Yf8Inr3/Qa05v8AuHkUn/CL6+P+X7S2/wC2DrUfVGbqVLzD/hGvEH/PbS//ACJTT4b8Reukf99yUvqT7i56Yf8ACLeIW/5f9Lg9xC8lR23gbWbZHWPxJa/O5c/8SoVpHC23RE3H7LJV8E6wT+98Vcf9M9MhWraeBIimLnW9Vl+jRpXS8PQ6RIjKSYv/AArvw83/AB8pe3P/AF1vZKZB4R8KWcwW10Gyc56zL5tZTqezjyodTU8j8W3Vvd+L9SayiiitYWFtEkWNuErFNVHYzGGmGtEhDDTTVAQy/db6VRrSOw0TiNSBS+WtTcTAIvpS7V/uigQn4UlACYooGaFOoZlcO9LUjGNVGf74rRBEv6Y+LW4U98/+g1StQDk/3UrLqzTctxxq3JzUyKOy0uhaJBtBwcZqG5X923/TJ8/g1KK1B7EujQeb9qb02r0q1DpzXOpyxSOscUPzcclhWNWfK2hxZdutNiikgZZTjzB1bBHeuklIZ9w6HkVxT1NDkvFYA1yU/wDPSCJ/qa5+f/j3eu6P8MzjuevTv+8X/rmn/oIrR0q42Q7fSvJZsa32j5eprktfk2eJfO/56W8f6ZFOGhDOk8D3ajW7iIn/AF0C45/uk131d+GnbQbCivQWpDiFFAKIUVNw5AqF7hV61jUnYrYzbi9JB7VkT3ws7e4vZT8lrEZj82OlebOfMyDwhGZ0Mr/ekYyH8TmkNekiWRmmGtCRhphqgI5fuN9Ko1cRFpfuClpMBKKAEpKEISkoKNGil5GQtLmpsMY3SqNx1FWkOMSxY/MGA71DD/qn9sUdTVEy3GFwRx6ipIw2OGxRsMRSRIMn5venea7RXIduqUluE2W9FnESycE4kDEDvxWl9r2yS3EMDpLKAn7xga5qsLyFFMQ3891E1o0Uf2h+Ipl+XbWvpdxJLpsL3CsJMbfmfJOOM1zVYpI1MLxY27Wsf3LZB+fNYE3+pP0NdUP4ZnE9Wmb/AFP/AFwi/wDQBU1lLtY15DNjUSbK1geKji50ybPXzIv61cY6ibNHwdcBfFVl8w/eRun47a9Ft7o9GIqua0iSz9pAPUVKs6kZNdUKw7h56Uxrle2Kv2wXIDe+hAoN58v3qx9qK5BJffTiqEt0T1rnnVuG5TefNcz45vPs/gy8TI33rLbL8wHfJpUviCWx5m1Qk16yMiJmpjSCrsIi8ykDbmxWgDZvuNVLvREZYG7aKXmk2ITJz0oJpgJmkzQAtJQI0KWhkMWikAxqp3FNbjRJYn5mH0pkP3nX+9xR1NbjghZsAZNWrf5o/lHAoewyKaTPFR5/d/WhIGT2Mmy42/8APX5fx7VfdnwRu2/hWU0ESjNcHD9/612FuRb2ECyFVjhhXeT0HFc1ZbIpvQ5C8uDd3U9y20GVs/KMcdqqSf6tvpXTy8sSYnqDP8sP/XCL/wBAFOR8NXkGty/FL8tZfio7rHT2/u3Tf+gVVPSQpIXwvLjxDorH/n4x+hr0XzttY1PiGPE9Si64+9U8zFYa13x3pj3PoaLsLEXne9Neb3oCxG81VpJaEiiuZOK4j4nX6KdH08MNyB7qT5/73Arpwy98iZwbXS/3s1EbivW5TMiM3tTDKatIBu800sfWnYRZfmI/SqlCEi2hwgHtTsipZIlJTGIaib71CAd2ooQGhRTIFoqWIRqp3HSnEB1o2N34Go+Qxwcc0dTUu2zxvMj71jbOCp702MTQEja68/3aXkVzDFxFOsjkB1+cKetRySvLJvfrT8yHuNq4t4HtxBdbioIw6dqUlctMmifTopVlaVpinIj8o03UdQkvsJjZbr0T1NY8jbuwbKZ6VG/+qOfQ1pLYIno4kzFAe3kR/wDoApQ/zV47RoXYpKreIPm0Xd/zzmVv51K0ZRmafcfZ2trhesVwG/WvT53xO/1p1kJDRJS+ZWKKGmWo/N5p2GL5tRmWmIa0tQs9ICPduYKOpryvx9fJfeNL5oSGig226ENuB2DFd2DXvGUzmzTa9QVwpKYgooETt/qv+A1WpIRYT7op1AhtFIBajfrTGKOlLQI0BS5pMzFp3akxkbVUuelODAS3OCc0zvTsago3VJukHSR/++qBWEwSc08REn7y0uYC3baY07hRcxgt0/dtWqPCwiiWa6vzsPaKNa5amJ5N0Xy32Gf2foELAPdzu3/TScR094vDCL88qH28+R6zviJaxBOO0kQfatBgYGK1SXHP+qasa5MG1vs/mheeJMZraCqW98bS6HbQSbrO1PrAn8qlD1wS0Y0yzE9Lqx3+H772Td+RFYx1kVJ6GD5uywkx1TLf1r1KSTPln+9Gjf8Ajoq661FEYZab5tYFgZKbvqrAIZKaZKAIzJTWk4osIIrlLWQ3UjAR2ymZiWx0rw+SUzSyTNndIxc5r0MF1MpbjKSvQJCkqhhTTU7iLLH92fpVansBYX7ooNSQFFMYLTG+9QMcKKTA0KdigyHU7FZyAhkqpP8AcrSI0RISKfu9qb1NBd9N3ehFJDF3t2NPw7D5Q/4UabjHBLj/AKbY/wB+m/Zieqr9TU3QtCRLRv7yj8KkWy9ZDn2WjnAlWwUn5t5HscVbg02zZQJLcnnr5hrKc3bQNTciAWJFUYCLtFSCuBlIdE9WJf3+n3MP9+Iis+prujm4jutnH95f6V6VY3IuNKsph/FAn6DFbV1fUlEhak3VylAXpu+gBN9MZ6LAR76TzKYGV4pvfsvhHUWz81xttlG7GcnJryuvUwa90yluJSV1CCkpiCkpjLDf6v8ACq9TcRMPu0UEC5paoYLUcn36gB46UUAaQp1DMxRT6kCB+tQtHv46VQDRbr70/wAgUXKuOWEDtUoj9l/Kk5BceI+1OEVQ5gP8upFjFS2Kw8JUoSsuYslVKsoPWspMETgVIBxWZpciZdjcdPSrdo25wP73FZMtPQ5u0OMj0JH613PhyT/imLMf3Gkj/I1pXJiXi9IXrnLGGTmgvQAzzKaZKBjfMpjSUEnL/EG7X7HptiGDNl7hwH6dhXE17GG0pmY2jNbMYlFMQGkpiLDf6v8ACq+aQiYfdpppCCnZ4qrgPSo3+/UgO7UUkBqYpaTMh1LUphcjIpMUXGLinYpgLinCplJFDqeKkSJMU4Cs7lkiipQtDETKKmUVi7FE61IKyKEZQeoyKihzFOi88EUeozAj+WVv95v5113heXOhsuR+7uX/AFrWutAiae6ml65CxN9IXpDE69KPKlPSNz+FJiuL5E46wyf981WuStuC9xIkK+rNTW5Nzh/Ekn9qazJcpLGkKosaF26gCsfyMXqQMQc45FezDSJndFv7FCg+ZGP1NKfIi4zGtVe5F7lK7ZHceWcjHpVariaIWkqgLDf6v8Kr1CAnA4oxQSLijFDkIVRUbfepIY4UU7gjUHel7VLMwo3UBYSkqbDHCnUALTxSmA6njrUbgPpwYVJRIrL2qQSCs5IqxLHJlgKtJWLCxMKkFSMKp6hIYbOadQCYl3YPehK7A5+S8hkuZJUygdt209q1tI8SabplvcRTR3MxkkDjyvpXZKg3Gwy0/jexH+q0q6b/AH5wKqv44/556Mn/AAK4zWf1TzHr1IofFusXt15NrBp8Hy7sumcVYbXdW24l1nTYm9IbUMar2EF0uQ3YpXHiC9Q/vtUvLj/rkiwis+fWbuUHbLdJn+JrglqpYeG6C7e5WtJ7iYt5tzO595DTJSWnO4Z28fNW1kIfGvmMCRVa6yt8exXGKtAhvI+ZiT+NN45IoGRN1pKYxKKYE7/6v8Kr0kBdx8o+lNNIgTNDcUIYzzKY0madhjMmk60DNsUUmYsSk70mwTFpaBi5FHmDFJjSuJ5oFKJaTHYUTP7flS72NQWoki7qcAfWsmUTAA1PGtQ7lEu8LINqge+KvDisrETJQ1PzSZIZqlqh/wCJTe/9cTThuM4qivTKCkNAx0OPN5q8COmcVDWpLZHKwB5qAsKaIH6d/EaW4P7yQnjeufxo6gTWf/HsP941Vuhu1AL67aVwGv8ApUTKQM1dxEbUlWWJRQBM33KhpCJjLxTC9FgG7jTaBjgrelL5dK4XHbRR0o6iNWkqSA7UUiRaXNMERTNxjtmm4oaNIjwKdWbZY4VIKW4EmTigSoPvMBWXKBKJPQE/Sl+1bXRTC3znb9/pS5BqRLmtUn5j9ayCQ4GnbqmxmG6qeoHdp9yvrGaqG4zjF6D6UteiWFIaQCK2Gre0jagkUhcuit8361NTYiRBfqjX8Ui8/OFP4VnXHF4/oTmnHUSH2TiOOZyuce9RTSeYwxnj1qkiupYsWz5i9lGRTHz/AGvH9VpE9SS4A8o4FVSSevSiIiKTrUdWUKabTGWWGVqvg+lSJDvLanCP1NADvLUU6gQlFAhKKQzRpM0XICl7UCCgnigEQyUvahmkRGl2HGKBMewpKIxRO2zPQ1GjylsMzU+VCJxHuq3G0MXDtj261NrkskF7aJ/z0/COoJ7uGaWIRiQfvB1FRyMFcu561pZrlkjaQ7dS7qixmBeq1yc20o9Y2/lVw3A48fdX6Utd6VzQWmmgVxnrWwby3jEbp+8dQMf7JxUyiKYj3CukbsGVs7sYrOf7wNEEJE+mn5Jwf9mmTqPMlKjvVbE9SS3+Vfc1GWzqkZ91pATSDcuPWofJpiILhNjDmoaEaIKKYFodKKRAUUXASkoGJRTASko6jNCkpGYU6hAAobmi4EL9KUUFDHG5+CKcsfrRcB6bAflYfnTXkw+fSiOoDfPPZiPpTPN57mqQWF3NjOMCow/79COzD+dSxm8/DNWhu4ri6ly2ANRup2Mw3VDKco3+6f5UIq5yg+6v0ortRSDtSUgEFThR5GD/AH80yZEt1d+cMbe/c1UJzQkESa1bCP8AWnnmkxdRBnHSotwF2rntQgJGuwPur+dM8+Z/9X/6DRawrEUgcEeZ1qOmiwpKYy2KWkiRKKVhDaWmIbSUrDCkzQBeFLQQwpwNGwgzQaLBYhfpQvShloYfv5pZD8tAxsWNxpz4204kkIU7qlEJzTbLJGj/AHTcE8dBVQY4I9RUgjoWO6Q/Wrea4+pUhM0bqqxmIWqNm5x68UJFHMDpRXYihM0UAJmpohvYbfm+lIknNhctyI9i+rnFRvDFEG3XSM391aXMIiikEYOVzmlNxIegCj6UxkfzueSaAB5oD/d70aDLCmFD8sWaDOx7YpE2K8rbjzUdMoSlqgLQ6UVBIUmaBCZopjGFhTS9CGM3UAMaYzTpc1JkJRnNHqIdRQBETmkzxQUN70yQE/MOtMBIUO/calbmn1GETY+lSs2CvBxSGXLL5PmPb7tVdRgX5pIU5bqq1lf3iTSBDHPtVkfdFc/U1lsNPWm1oZiGoyfnX600I531+ppK6ehpcSikUPgk8m4SXarbTnDDirTajeTcR4A/6Zx0mu5DI/s1xN88p/76NKbNUUlpO3pii4uYqin1RQ+Oo3/1lStwHgU7ZTuBBNw1MqgQUgpCLY6UVIhtJuFMBhemF6Cg2s1PEXrQS2OCgUtIC1S0yQp2aHqSFFIRG/SmChFodjNNLKP4h+dMQqHjjmnGORvalcuxJHFtXDMDUwdEGCcVnrIq437TEnfP0FN+2kHKRc47mn7LS4i+Ox9RmrS/cX6VgOQhppoWhA2mfxD61QHOtxn6mm10lhRQA+22/aY/M+5u5rQN2n/LOE59elKUbkvUja4lP8eP90YqBuQetA7WKy06qKJYehpn/L0PripsSWOlNBpCILj71RVZQUnemIsikJwKkRCXoAZugNAx4h9TUgUCkFwPAphf0oJG/MaVOtAF2lp9QCilckWkNFwGmmr0pjHeV5hx5bP7LVq306Tn/R2i92GanmW1xvYmeLY20fyqMrTsY8wxO496iuFwalKxoV9tSbfSqk9B3NJD+7TH90CrkfMS1yGrAimkVSMxpqLPzA+9UgMCUYc/WmV02KQtIaSAVPvCrFBQtIec4qGIrClPStBklv0amsMTipJ6k9JRYCvN96oqsYUooAs0x/u1BAyEZzViqY2MLYpm5j7VIhNtLVDG0qUCP//Z'
img_data = b64decode(base)
img_buffer = BytesIO(img_data)
img_buffer.seek(0)
image_src = Image.open(img_buffer)

#image_src = Image.open(input_image)
# resize image
# size = 640, 480
# size = 320, 240
# size = 250, 170
# size = 160, 120
# image_src.thumbnail(size, Image.ANTIALIAS)
    
imgByteArr = io.BytesIO()

image_src.save(imgByteArr, format=image_src.format)
imgByteArr = imgByteArr.getvalue()

# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=imgByteArr)
result = response['Body'].read().decode()
res = json.loads(result) # convert json string to Python dict for parsing

Display Detectron2 Panoptic segmentation inference results

In [ ]:
image_src = image_src.convert('RGBA')

# Define function inputs 
opacity=150
panoptic_seg=np.array(res[0]['panoptic_seg']) #res[0]['panoptic_seg'].cpu().numpy()
instance_list=res[0]['instance_list']
boxes=res[0]['pred_boxes']
# Call function
out=pan_seg_visualizer(panoptic_seg, instance_list, image_src, metadata["stuff_classes"], metadata["stuff_colors"], metadata["thing_classes"], metadata["thing_colors"], boxes, opacity)
out.show()

### Call Panoptic API using Python Request library

Create AWS API gateway before performing this step

In [ ]:
# import some common libraries
# Using Python Request library
import requests
import json
import numpy as np
import time
import io
from PIL import Image, ImageDraw, ImageFont

# Define Constants
API_INVOKE_URL="<INSERT_API_INVOKE_URL_HERE>"

# define variables
url=API_INVOKE_URL

def cloud_api_predict(headers, payload):
    # send POST request to url
    return requests.request("POST", url, headers=headers, data=payload).text

# Read image into memory - needed because of image size reduction
image_src = Image.open(input_image)

imgByteArr = io.BytesIO()

image_src.save(imgByteArr, format=image_src.format)
payload = imgByteArr.getvalue()

headers = {
  'Accept': 'image/jpeg',
  'Content-Type': 'image/jpeg'
}

predictions=cloud_api_predict(headers, payload)
pred_dict=json.loads(predictions) # convert json string to Python dict for parsing

Display Detectron2 Panoptic inference results from API

In [ ]:
image_src = image_src.convert('RGBA')

# Define function inputs 
opacity=150
panoptic_seg=np.array(pred_dict[0]['panoptic_seg'])
instance_list=pred_dict[0]['instance_list']
boxes=pred_dict[0]['pred_boxes']
# Call function
out=pan_seg_visualizer(panoptic_seg, instance_list, image_src, metadata["stuff_classes"], metadata["stuff_colors"], metadata["thing_classes"], metadata["thing_colors"], boxes, opacity)
out.show()